In [2]:
import pandas as pd

known_gene = pd.read_csv("knownGene.txt", sep="\t", header=None)
known_canonical = pd.read_csv("knownCanonical2.txt", sep="\t", header=None)

canonical_transcripts = set(known_canonical[4])

filtered_known_gene = known_gene[known_gene[0].isin(canonical_transcripts)]

filtered_known_gene.to_csv("filtered_knownGene.txt", sep="\t", index=False, header=False)


In [3]:
import pandas as pd

known_gene = pd.read_csv("knownGene.txt", sep="\t", header=None)
known_canonical = pd.read_csv("knownCanonical2.txt", sep="\t", header=None)

known_gene_transcripts = known_gene[0].unique()
known_canonical_transcripts = known_canonical[4].unique()

known_gene_set = set(known_gene_transcripts)
known_canonical_set = set(known_canonical_transcripts)

print("Distinct transcripts in knownGene:", len(known_gene_set))
print("Distinct transcripts in knownCanonical:", len(known_canonical_set))

common_transcripts = known_gene_set.intersection(known_canonical_set)
only_in_known_gene = known_gene_set.difference(known_canonical_set)
only_in_known_canonical = known_canonical_set.difference(known_gene_set)

print("\nNumber of common transcripts:", len(common_transcripts))
print("Number of transcripts only in knownGene:", len(only_in_known_gene))
print("Number of transcripts only in knownCanonical:", len(only_in_known_canonical))


Distinct transcripts in knownGene: 254491
Distinct transcripts in knownCanonical: 64955

Number of common transcripts: 64955
Number of transcripts only in knownGene: 189536
Number of transcripts only in knownCanonical: 0


In [5]:
unique_transcripts = filtered_known_gene[0].unique()

num_unique_transcripts = len(unique_transcripts)

print("Number of unique transcripts in filtered_knownGene.txt:", num_unique_transcripts)

Number of unique transcripts in filtered_knownGene.txt: 64955


In [7]:
# The chromosome column is the second column (index 1)
distinct_chromosomes = filtered_known_gene[1].unique()

# Print the list of distinct chromosome names
print("Distinct chromosome names:")
print(distinct_chromosomes)

Distinct chromosome names:
['chr1' 'chr10' 'chr11' 'chr12' 'chr13' 'chr14' 'chr15' 'chr16' 'chr17'
 'chr18' 'chr19' 'chr2' 'chr20' 'chr21' 'chr22' 'chr3' 'chr4' 'chr5'
 'chr6' 'chr7' 'chr8' 'chr9' 'chrM' 'chrMT' 'chrX' 'chrY']


ignore the above, that was just data exploration. BELOW:Filter the knownGene dataset so that it only includes transcripts present in the knownCanonical2.txt list (the canonical transcripts).
Further filter out non-coding RNA types like pseudo, lncRNA, and lincRNA based on the gene type.
Exclude rows with irregular chromosomes (e.g., chrM, chrMT).

filter out any non coding regions :

In [18]:
import pandas as pd

known_gene_bed = pd.read_csv("knownGene.bed", sep="\t", header=None)
known_canonical = pd.read_csv("knownCanonical2.txt", sep="\t", header=None)

# Extract the canonical transcript IDs (5th column of knownCanonical2.txt)
canonical_transcripts = set(known_canonical[4])

# Filter knownGene.bed to include only canonical transcripts (4th column of knownGene.bed)
filtered_known_gene = known_gene_bed[known_gene_bed[3].isin(canonical_transcripts)]

# List of standard chromosomes (excluding mitochondrial and irregular chromosomes)
standard_chromosomes = {f"chr{i}" for i in range(1, 23)}.union({"chrX", "chrY"})

# Filter out rows with irregular chromosome types (e.g., 'chrM', 'chrMT')
filtered_no_irregular_chromosomes = filtered_known_gene[filtered_known_gene[0].isin(standard_chromosomes)]

# Save the filtered DataFrame to a new file
filtered_no_irregular_chromosomes.to_csv("filtered_knownGene_noIrregular.txt", sep="\t", index=False, header=False)

# Display some stats
num_filtered_rows = filtered_no_irregular_chromosomes.shape[0]
distinct_chromosomes = filtered_no_irregular_chromosomes[0].unique()

print(f"Number of rows in the filtered file: {num_filtered_rows}")
print("Distinct chromosome names after filtering:")
print(distinct_chromosomes)
print(known_gene_bed.

Number of rows in the filtered file: 64926
Distinct chromosome names after filtering:
['chr1' 'chr10' 'chr11' 'chr12' 'chr13' 'chr14' 'chr15' 'chr16' 'chr17'
 'chr18' 'chr19' 'chr2' 'chr20' 'chr21' 'chr22' 'chr3' 'chr4' 'chr5'
 'chr6' 'chr7' 'chr8' 'chr9' 'chrX' 'chrY']
     0      1      2                    3   4  5      6      7         8   9   \
0  chr1  11868  14409  ENST00000456328.2_1   0  +  11868  11868     25600   3   
1  chr1  12009  13670  ENST00000450305.2_2   0  +  12009  12009  16724991   6   
2  chr1  14695  24886  ENST00000488147.2_5   0  -  14695  14695  16724991  10   
3  chr1  29553  31097  ENST00000473358.1_5   0  +  29553  29553     25600   3   
4  chr1  30266  31109  ENST00000469289.1_1   0  +  30266  30266     25600   2   

   ...           17    18                                  19         20  \
0  ...      DDX11L2  none                              lncRNA  nonCoding   
1  ...      DDX11L1  none  transcribed_unprocessed_pseudogene     pseudo   
2  ...       W

In [16]:
import pandas as pd

# Load the filtered file without irregular chromosomes
filtered_no_irregular = pd.read_csv("filtered_knownGene_noIrregular.txt", sep="\t", header=None)

# The gene type column appears to be the 23rd column (index 22)
# Filter out rows where the gene type is 'pseudo' or 'lncRNA'
# Define the list of gene types to keep
keep_gene_types = [
    'protein_coding']

# Filter the DataFrame to keep only rows with the specified gene types
filtered_final = filtered_no_irregular[filtered_no_irregular[22].isin(keep_gene_types)]

# Save the final filtered DataFrame to a new file
filtered_final.to_csv("filtered_knownGene_final.txt", sep="\t", index=False, header=False)

# Display some statistics
num_final_rows = filtered_final.shape[0]
distinct_final_gene_types = filtered_final[22].unique()

print(f"Number of rows in the final filtered file: {num_final_rows}")
print("Distinct gene types remaining in the final filtered file:")
print(distinct_final_gene_types)

Number of rows in the final filtered file: 19810
Distinct gene types remaining in the final filtered file:
['protein_coding']


ok so i did this: 
filter the knownGene dataset so that it only includes transcripts present in the knownCanonical
- filter out non-coding RNA types like pseudo, lncRNA, and lincRNA etc
-Exclude rows with irregular chromosomes (chrM, chrMT).
Number of rows in the final filtered file: 26623
Distinct gene types remaining in the final filtered file:
['protein_coding']

In [17]:
print(filtered_final.head())

      0       1       2                    3   4  5       6       7       8   \
8   chr1   65418   71585  ENST00000641515.2_5   0  +   65564   70008  789624   
12  chr1  134900  139379    ENST00000423372.3   0  -  138529  139309  789624   
24  chr1  367639  368634    ENST00000426406.1   0  +  367658  368597  789624   
40  chr1  621095  622034  ENST00000332831.5_7   0  -  621095  622034  789624   
63  chr1  859302  879954  ENST00000616016.5_6   0  +  859811  879533  789624   

    9   ...          17          18              19      20  \
8    3  ...       OR4F5  A0A2U3U0J3  protein_coding  coding   
12   2  ...  AL627309.1      R4GN28  protein_coding  coding   
24   1  ...      OR4F29      Q6IEY1  protein_coding  coding   
40   1  ...      OR4F16      Q6IEY1  protein_coding  coding   
63  14  ...      SAMD11  A0A087WU74  protein_coding  coding   

                                        21              22  \
8                      havana_homo_sapiens  protein_coding   
12              

In [15]:
reference_genome = pysam.FastaFile("hg19.fa")


In [16]:
import pybedtools
import pandas as pd
import pandas as pd
import pybedtools
import pysam
import random
from collections import Counter
# Load the BED file using only the first 12 columns
bed_file = pybedtools.BedTool(pybedtools.BedTool("filtered_knownGene_final.txt").cut([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))

# Reference genome file (e.g., hg38.fa)
reference_genome = pysam.FastaFile("hg19.fa")

# extract sequences using PyBedTools' built-in sequence function
bed_with_sequences = bed_file.sequence(fi="hg19.fa", s=True)

#read the resulting sequences from the output file generated by PyBedTools
sequences = []
with open(bed_with_sequences.seqfn) as seq_file:
    transcript_id = ""
    for line in seq_file:
        if line.startswith(">"):
            transcript_id = line.strip().split()[0][1:]  
        else:
            sequence = line.strip()
            sequences.append({"transcript_id": transcript_id, "sequence": sequence})

# Convert to DataFrame and save
df_sequences = pd.DataFrame(sequences)
df_sequences.to_csv("extracted_transcript_sequences_pybedtools.txt", sep="\t", index=False)

print("Sequence extraction complete. Results saved to 'extracted_transcript_sequences_pybedtools.txt'.")


Sequence extraction complete. Results saved to 'extracted_transcript_sequences_pybedtools.txt'.


In [4]:
print(bed_file.head())

chr1	65418	71585	ENST00000641515.2_5	0	+	65564	70008	789624	3	15,54,2549,	0,101,3618,
 chr1	134900	139379	ENST00000423372.3	0	-	138529	139309	789624	2	902,1759,	0,2720,
 chr1	367639	368634	ENST00000426406.1	0	+	367658	368597	789624	1	995,	0,
 chr1	621095	622034	ENST00000332831.5_7	0	-	621095	622034	789624	1	939,	0,
 chr1	859302	879954	ENST00000616016.5_6	0	+	859811	879533	789624	14	1026,92,182,51,125,90,138,163,116,79,500,125,111,667,	0,1999,6232,7116,11849,15117,15352,17221,18213,18487,18636,19330,19775,19985,
 chr1	879582	894636	ENST00000327044.7_6	0	-	880073	894620	789624	19	598,90,136,114,144,102,114,112,140,189,114,111,79,91,121,132,175,153,42,	0,854,1315,1970,2199,3928,4287,6924,7797,8209,8972,9579,9801,11720,11892,12691,12896,14726,15012,
 chr1	895963	901099	ENST00000338591.8_3	0	+	896073	900571	789624	12	217,260,122,222,117,214,145,168,89,74,182,757,	0,709,1045,1242,1771,2120,2525,2753,3336,3523,3765,4379,
 chr1	901861	911245	ENST00000379410.8_9	0	+	901911	909955	789624	16	133,

In [5]:
# Helper function to get the reverse complement of a sequence
def reverse_complement(sequence):
    complement = str.maketrans("ATCGatcg", "TAGCtagc")
    return sequence.translate(complement)[::-1]

exon_sequences = []
intron_sequences = []

# Process each BED record
for bed_record in bed_file:
    chrom = bed_record.chrom
    strand = bed_record.strand
    chrom_start = int(bed_record.start)  # Transcription start
    chrom_end = int(bed_record.end)  # Transcription end
    exon_lengths = list(map(int, bed_record.fields[10].strip(',').split(',')))  # Exon lengths
    exon_offsets = list(map(int, bed_record.fields[11].strip(',').split(',')))  # Exon start offsets

    # Calculate absolute exon starts and ends
    exon_starts = [chrom_start + offset for offset in exon_offsets]
    exon_ends = [start + length for start, length in zip(exon_starts, exon_lengths)]

    previous_end = chrom_start

    for exon_start, exon_end in zip(exon_starts, exon_ends):
        # Get the exon sequence
        exon_seq = reference_genome.fetch(chrom, exon_start, exon_end)
        if strand == '-':
            exon_seq = reverse_complement(exon_seq)
        exon_sequences.append({"transcript_id": bed_record.name, "sequence": exon_seq})

        if previous_end < exon_start:
            intron_seq = reference_genome.fetch(chrom, previous_end, exon_start)
            if strand == '-':
                intron_seq = reverse_complement(intron_seq)
            intron_sequences.append({"transcript_id": bed_record.name, "sequence": intron_seq})

        previous_end = exon_end

    if previous_end < chrom_end:
        intron_seq = reference_genome.fetch(chrom, previous_end, chrom_end)
        if strand == '-':
            intron_seq = reverse_complement(intron_seq)
        intron_sequences.append({"transcript_id": bed_record.name, "sequence": intron_seq})

df_exons = pd.DataFrame(exon_sequences)
df_exons.to_csv("extracted_exon_sequences.txt", sep="\t", index=False)

df_introns = pd.DataFrame(intron_sequences)
df_introns.to_csv("extracted_intron_sequences.txt", sep="\t", index=False)

print("Exon and intron sequence extraction complete.")


Exon and intron sequence extraction complete.


In [3]:
import pandas as pd

intron_file = "extracted_intron_sequences.txt"
fraction_to_extract = 0.01  # Extract the first 1/10th

total_rows = sum(1 for _ in open(intron_file)) - 1  # Exclude header row
rows_to_extract = int(total_rows * fraction_to_extract)

# Read the first 1/10th of the file
df_introns = pd.read_csv(intron_file, sep="\t", nrows=rows_to_extract)

df_introns.to_csv("extracted_intron_sequences_subset.txt", sep="\t", index=False)

print(f"Extracted {rows_to_extract} rows out of {total_rows}. Results saved to 'extracted_intron_sequences_subset.txt'.")


Extracted 1864 rows out of 186430. Results saved to 'extracted_intron_sequences_subset.txt'.


In [70]:
import pandas as pd

canonical_dataset_file = "canonical_dataset_created.txt"

query_gene_name = "P11182"
query_chrom = "chr1"
query_strand = "-"
query_chromStart = 100652475
query_chromEnd = 100715390
query_exonStarts = {100652475, 100671786, 100672001, 100676250, 100680373, 100681539, 100684182, 100696289, 100700992, 100706317, 100715326}
query_exonEnds = {100661979, 100671858, 100672193, 100676328, 100680540, 100681756, 100684304, 100696471, 100701068, 100706441, 100715391}

# load canonical dataset
canonical_df = pd.read_csv(canonical_dataset_file, sep="\t", header=None, names=[
    "geneName", "chrom", "strand", "chromStart", "chromEnd", "exonEnds", "exonStarts"
])

# Ensure exonStarts and exonEnds are processed into sets for comparison
canonical_df["exonStartsSet"] = canonical_df["exonStarts"].apply(lambda x: set(map(int, x.split(","))))
canonical_df["exonEndsSet"] = canonical_df["exonEnds"].apply(lambda x: set(map(int, x.split(","))))

matches = canonical_df[
    (canonical_df["geneName"] == query_gene_name) &
    (canonical_df["chrom"] == query_chrom) &
    (canonical_df["strand"] == query_strand) &
    (canonical_df["chromStart"] == query_chromStart) &
    (canonical_df["chromEnd"] == query_chromEnd) &
    (canonical_df["exonStartsSet"] == query_exonStarts) &
    (canonical_df["exonEndsSet"] == query_exonEnds)
]

# Output the result
if not matches.empty:
    print("The canonical dataset contains an exact match for the transcript:")
    print(matches)
else:
    print("No exact match for the transcript found in the canonical dataset.")


The canonical dataset contains an exact match for the transcript:
    geneName chrom strand  chromStart   chromEnd  \
925   P11182  chr1      -   100652475  100715390   

                                              exonEnds  \
925  100661979,100671858,100672193,100676328,100680...   

                                            exonStarts  \
925  100652475,100671786,100672001,100676250,100680...   

                                         exonStartsSet  \
925  {100700992, 100672001, 100696289, 100681539, 1...   

                                           exonEndsSet  
925  {100672193, 100676328, 100706441, 100701068, 1...  


In [113]:
import pandas as pd

file_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_with_flanked_Ns.txt"

query_start = 100652475 - 5001  # Adjust for flanking N's
query_end = 100715390 + 5000
query_chrom = "chr1"

# Read the file and check for matching coordinates
matches = []
with open(file_path, "r") as f:
    for line in f:
        if line.startswith(">"):  # Check only header lines
            header = line.strip().lstrip(">")
            chrom, region = header.split(":")
            region_start, region_end = map(int, region.split("-"))
            
            
            if chrom == query_chrom and region_start == query_start and region_end == query_end:
                matches.append(header)


if matches:
    print("Matching headers found:")
    for match in matches:
        print(match)
else:
    print("No matching headers found.")


No matching headers found.


In [17]:
def calculate_intron_frequencies(intron_sequences):
    counts = Counter(''.join(intron_sequences))
    total = sum(counts.values())
    frequencies = {
        'A': counts.get('A', 0) / total,
        'T': counts.get('T', 0) / total,
        'C': counts.get('C', 0) / total,
        'G': counts.get('G', 0) / total
    }
    return frequencies

def generate_random_sequence(length, frequencies):
    nucleotides = ['A', 'T', 'C', 'G']
    probs = [frequencies[nuc] for nuc in nucleotides]
    return ''.join(random.choices(nucleotides, probs, k=length))

In [18]:
# Helper function to get the reverse complement of a sequence
def reverse_complement(sequence):
    complement = str.maketrans("ATCGatcg", "TAGCtagc")
    return sequence.translate(complement)[::-1]

# Extract intron sequences and calculate overall intron nucleotide frequencies
intron_sequences = []

for bed_record in bed_file:
    chrom = bed_record.chrom
    strand = bed_record.strand
    chrom_start = int(bed_record.start)
    chrom_end = int(bed_record.end)
    exon_starts = list(map(int, bed_record.fields[10].strip(',').split(',')))
    exon_ends = list(map(int, bed_record.fields[11].strip(',').split(',')))

    previous_end = chrom_start
    for start_offset, end_offset in zip(exon_starts, exon_ends):
        exon_start = chrom_start + start_offset
        exon_end = chrom_start + end_offset

        # Get intron sequence before the current exon
        if previous_end < exon_start:
            intron_seq = reference_genome.fetch(chrom, previous_end, exon_start)
            if strand == '-':
                intron_seq = reverse_complement(intron_seq)
            intron_sequences.append(intron_seq)

        previous_end = exon_end

    # Add the last intron sequence if it exists
    if previous_end < chrom_end:
        intron_seq = reference_genome.fetch(chrom, previous_end, chrom_end)
        if strand == '-':
            intron_seq = reverse_complement(intron_seq)
        intron_sequences.append(intron_seq)


In [19]:
import pybedtools
import pandas as pd
import pandas as pd
import pybedtools
import pysam
import random
from collections import Counter
intron_frequencies = calculate_intron_frequencies(intron_sequences)


In [20]:
# Function to process each transcript by generating random exons
def process_transcript(transcript_id, full_sequence, bed_record):
    chrom = bed_record.chrom
    strand = bed_record.strand
    chrom_start = int(bed_record.start)
    exon_starts = list(map(int, bed_record.fields[10].strip(',').split(',')))
    exon_ends = list(map(int, bed_record.fields[11].strip(',').split(',')))

    processed_sequence = []
    previous_end = 0

    for start_offset, end_offset in zip(exon_starts, exon_ends):
        exon_start = chrom_start + start_offset
        exon_end = chrom_start + end_offset

        # Add the intron sequence before the exon
        if previous_end < start_offset:
            intron_seq = full_sequence[previous_end:start_offset]
            processed_sequence.append(intron_seq)

        exon_length = exon_end - exon_start
        random_exon = generate_random_sequence(exon_length, intron_frequencies)
        processed_sequence.append(random_exon)

        previous_end = end_offset

    if previous_end < len(full_sequence):
        processed_sequence.append(full_sequence[previous_end:])

    final_sequence = ''.join(processed_sequence)
    padded_sequence = f"{padding_sequence}{final_sequence}{padding_sequence}"

    fasta_header = f">{transcript_id}|{chrom}|{strand}|{chrom_start}|{previous_end}"
    return fasta_header, padded_sequence

with open("canonical_dataset_spliceai.fasta", "w") as output_file:
    for idx, row in df_sequences.iterrows():
        transcript_id = row["transcript_id"]
        full_sequence = row["sequence"]
        bed_record = bed_file[idx]
        header, padded_sequence = process_transcript(transcript_id, full_sequence, bed_record)
        output_file.write(f"{header}\n{padded_sequence}\n")

print("Canonical dataset preparation complete. Saved to 'canonical_dataset_spliceai.fasta'.")


Canonical dataset preparation complete. Saved to 'canonical_dataset_spliceai.fasta'.



example row:
>chr1:65418-71585(+)|chr1|+|65418|3618
chr1:65418-71585(+):

The transcript is located on chromosome chr1.
The genomic range spans from 65418 to 71585.
The transcript is on the + strand (forward strand).
|chr1|:

Confirms the chromosome (chr1).
+:

Strand orientation is positive (+).
65418:

Start position of the transcript on the chromosome.
3618:

End position of the last exon, relative to the transcript start (chromStart). This value might indicate the last position of the exon offset or an internal boundary within the transcript.

In [101]:
# import pysam

# # Paths
# reference_genome_path = "/mnt/lareaulab/sdahiyat/hg19.fa"  # Reference genome
# canonical_fasta_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_dataset_spliceai.fasta"
# output_file_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_with_flanked_Ns.txt"


# reference_genome = pysam.FastaFile(reference_genome_path)

# with open(canonical_fasta_path, "r") as fasta_file, open(output_file_path, "w") as output_file:
#     for line in fasta_file:
#         if line.startswith(">"):  # Header line
#             header = line.strip().lstrip(">")
#             chrom, region_strand = header.split("|")[0].split(":")
#             strand = region_strand[-2]  # "+" or "-"
#             region = region_strand[:-3]  # Extract region without strand info
#             chrom_start, chrom_end = map(int, region.split("-"))

#             adjusted_start = max(0, chrom_start - 5001)  # Avoid negative values
#             adjusted_end = chrom_end + 5000

#             sequence = reference_genome.fetch(chrom, chrom_start, chrom_end)
#             if strand == "-":  # Reverse complement for negative strand
#                 sequence = sequence.translate(str.maketrans("ATCG", "TAGC"))[::-1]

#             padded_sequence = f"{'N' * 5001}{sequence}{'N' * 5000}"

#             output_file.write(f"{chrom}:{adjusted_start}-{adjusted_end}\t{padded_sequence}\n")

# print(f"Canonical sequence file with flanked 'N's created at {output_file_path}")


In [17]:
import pandas as pd

input_file = "/mnt/lareaulab/sdahiyat/filtered_knownGene_final.txt"
columns = [
    "chrom", "chromStart", "chromEnd", "name", "score", "strand",
    "thickStart", "thickEnd", "itemRgb", "blockCount", "blockSizes", "blockStarts",
]
df = pd.read_csv(input_file, sep="\t", header=None, names=columns)

# Find problematic rows
invalid_rows = df[~df["chromStart"].apply(str.isdigit) | ~df["chromEnd"].apply(str.isdigit)]
print("Invalid rows:")
print(invalid_rows)
print(len(invalid_rows))

Invalid rows:
                                                                                                                                                                                                                              chrom  \
chr1 65418    71585    ENST00000641515.2_5 0 + 65564    70008    789624 3  15,54,2549,                                        0,101,3618,                                        uc328hnl.1 none none                       -1,0,0,   
     134900   139379   ENST00000423372.3   0 - 138529   139309   789624 2  902,1759,                                          0,2720,                                            uc319sre.1 none none                         -1,0,   
     367639   368634   ENST00000426406.1   0 + 367658   368597   789624 1  995,                                               0,                                                 uc319vnl.1 none none                            0,   
     621095   622034   ENST00000332831.5_7 0 - 621095   622034

In [4]:
#how to make canonical sequences file


import pysam
import pandas as pd

# Paths
reference_genome_path = "/mnt/lareaulab/sdahiyat/hg19.fa"  # Reference genome
canonical_dataset_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_dataset_created.txt"
output_file_path = "/mnt/lareaulab/sdahiyat/illumina/canonical_sequence_with_flanked_Ns.txt"

reference_genome = pysam.FastaFile(reference_genome_path)

canonical_df = pd.read_csv(
    canonical_dataset_path,
    sep="\t",

    header=None,
    names=["geneName", "chrom", "strand", "chromStart", "chromEnd", "exonEnds", "exonStarts"]
)

# Create the canonical sequence file with flanked 'N's
with open(output_file_path, "w") as output_file:
    for _, row in canonical_df.iterrows():
        chrom = row["chrom"]
        chrom_start = row["chromStart"]
        chrom_end = row["chromEnd"]
        strand = row["strand"]

        adjusted_start = max(0, chrom_start-5001)  #  or chrom_start-5001
        adjusted_end = chrom_end +5000 #or add 5000

        core_sequence = reference_genome.fetch(chrom, chrom_start-1, chrom_end)  # Use adjusted coordinates
        # if strand == "-":  # Reverse complement for negative strand
        #      core_sequence = core_sequence.translate(str.maketrans("ATCG", "TAGC"))[::-1]

        padded_sequence = f"{'N' * 5000}{core_sequence}{'N' * 5000}"

        output_file.write(f"{chrom}:{adjusted_start}-{adjusted_end}\t{padded_sequence}\n")

print(f"Canonical sequence file with flanked 'N's created at {output_file_path}")


Canonical sequence file with flanked 'N's created at /mnt/lareaulab/sdahiyat/illumina/canonical_sequence_with_flanked_Ns.txt


In [12]:
import h5py

file_path = "/mnt/lareaulab/sdahiyat/illumina/datafile_train_0.h5"

with h5py.File(file_path, "r") as h5f:
    print("Datasets in the file:", list(h5f.keys()))
    
    # Inspect contents of each dataset
    for dataset_name in h5f.keys():
        data = h5f[dataset_name][:]
        print(f"\nDataset: {dataset_name}")
        print(f"Data: {data[:10]}") 


Datasets in the file: []


In [18]:
import pandas as pd

file_path = "/mnt/lareaulab/sdahiyat/filtered_knownGene_final.txt"
df = pd.read_csv(file_path, sep="\t", header=None)
print("First few rows of the dataset:")
print(df.head())
print("\nDataset shape:", df.shape)


First few rows of the dataset:
     0       1       2                    3   4  5       6       7       8   \
0  chr1   65418   71585  ENST00000641515.2_5   0  +   65564   70008  789624   
1  chr1  134900  139379    ENST00000423372.3   0  -  138529  139309  789624   
2  chr1  367639  368634    ENST00000426406.1   0  +  367658  368597  789624   
3  chr1  621095  622034  ENST00000332831.5_7   0  -  621095  622034  789624   
4  chr1  859302  879954  ENST00000616016.5_6   0  +  859811  879533  789624   

   9   ...          17          18              19      20  \
0   3  ...       OR4F5  A0A2U3U0J3  protein_coding  coding   
1   2  ...  AL627309.1      R4GN28  protein_coding  coding   
2   1  ...      OR4F29      Q6IEY1  protein_coding  coding   
3   1  ...      OR4F16      Q6IEY1  protein_coding  coding   
4  14  ...      SAMD11  A0A087WU74  protein_coding  coding   

                                       21              22  \
0                     havana_homo_sapiens  protein_coding   

In [1]:
import pandas as pd

input_file = "filtered_knownGene_final.txt"
output_file = "illumina/canonical_dataset_created.txt"

df = pd.read_csv(input_file, sep="\t", header=None)

df.columns = [
    "chrom", "chromStart", "chromEnd", "name", "score", "strand",
    "thickStart", "thickEnd", "itemRgb", "blockCount",
    "blockSizes", "blockStarts", "other1", "other2", "other3",
    "other4", "other5", "other6", "geneName", "other9",
    "geneType", "typeCategory", "extraInfo", "category",
    "canonicalStatus", "placeholder", "additional"
]

df["chromStart"] = pd.to_numeric(df["chromStart"], errors="coerce")
df["chromEnd"] = pd.to_numeric(df["chromEnd"], errors="coerce")
df["blockSizes"] = df["blockSizes"].str.rstrip(",").str.split(",").apply(lambda x: list(map(int, x)))
df["blockStarts"] = df["blockStarts"].str.rstrip(",").str.split(",").apply(lambda x: list(map(int, x)))

results = []

for _, row in df.iterrows():
    try:
        chrom = row["chrom"]
        strand = row["strand"]
        gene_name = row["geneName"]
        chrom_start = row["chromStart"] + 1
        chrom_end = row["chromEnd"]
        block_sizes = row["blockSizes"]
        block_starts = row["blockStarts"]

        exon_ends = [chrom_start + start + size - 1 for start, size in zip(block_starts, block_sizes)]  # Adjusting exon ends
        exon_starts = [chrom_start + start for start in block_starts]
        exon_ends = exon_ends[:-1] 
        exon_starts = exon_starts[1:] 
        if len(exon_ends) != len(exon_starts):
            print(f"Warning: Unequal exon ends and starts for gene {gene_name}. Skipping.")
            continue

        exon_ends_str = ",".join(map(str, exon_ends))
        exon_starts_str = ",".join(map(str, exon_starts))

        results.append([gene_name,0, chrom, strand, chrom_start, chrom_end, exon_ends_str, exon_starts_str])
    except Exception as e:
        print(f"Error processing row: {row}")
        print(e)

output_df = pd.DataFrame(results, columns=["geneName","paralog", "chrom", "strand", "chromStart", "chromEnd", "exonEnds", "exonStarts"])

output_df.to_csv(output_file, sep="\t", index=False, header=False)

print(f"Canonical dataset saved to {output_file}")


Canonical dataset saved to illumina/canonical_dataset_created.txt


In [11]:
import h5py

datafile_path = './illumina/datafile_train_all.h5'  # Adjust the path if needed

with h5py.File(datafile_path, 'r') as h5f:
    print("Datasets in the file:")
    for key in h5f.keys():
        print(f"{key}: {h5f[key].dtype}, shape={h5f[key].shape}")


Datasets in the file:


In [61]:
import pandas as pd


created_file = "canonical_dataset_created.txt"
query_data = 	[
    {"txEnd": 100715390, "geneName": "DBT"},
    {"txEnd": 8404225, "geneName": "SLC45A1"},
    {"txEnd": 16786573, "geneName": "NECAP2"},
]

columns_created = ["geneName", "chrom", "strand", "chromStart", "chromEnd", "exonEnds", "exonStarts"]
df_created = pd.read_csv(created_file, sep="\t", header=None, names=columns_created)

df_created["chromEnd"] = pd.to_numeric(df_created["chromEnd"], errors="coerce")

matches = []
for query in query_data:
    tx_end, gene_name = query["txEnd"], query["geneName"]
    match = df_created[df_created["chromEnd"] == tx_end]
    if not match.empty:
        matches.append({"queryGene": gene_name, "matchedRows": match})

for match in matches:
    print(f"Query Gene: {match['queryGene']}")
    print(match["matchedRows"].to_string(index=False))
    print("\n")

output_file = "matched_rows_by_txend.txt"
with open(output_file, "w") as f:
    for match in matches:
        f.write(f"Query Gene: {match['queryGene']}\n")
        f.write(match["matchedRows"].to_string(index=False))
        f.write("\n\n")

print(f"Matches saved to {output_file}")


Query Gene: DBT
geneName chrom strand  chromStart  chromEnd                                                                                                      exonEnds                                                                                                    exonStarts
  P11182  chr1      -   100652475 100715390 100661979,100671858,100672193,100676328,100680540,100681756,100684304,100696471,100701068,100706441,100715391 100652475,100671786,100672001,100676250,100680373,100681539,100684182,100696289,100700992,100706317,100715326


Query Gene: SLC45A1
geneName chrom strand  chromStart  chromEnd                                                                exonEnds                                                              exonStarts
  Q9Y2W3  chr1      +     8378174   8404225 8378247,8384787,8385451,8386103,8390997,8395651,8398053,8399759,8404226 8378174,8384366,8385358,8385878,8390269,8395497,8397876,8399553,8403807


Query Gene: NECAP2
geneName chrom strand  chromStart  ch

In [13]:
import pandas as pd

filtered_known_gene = pd.read_csv("filtered_knownGene_final.txt", sep="\t", header=None)
gene_names = filtered_known_gene.iloc[:, 18].unique()  # Assuming 'geneName' is column 19
print(gene_names)


['A0A2U3U0J3' 'R4GN28' 'Q6IEY1' ... 'Q9NR90' 'A0A804HKV4' 'A0A024R061']


In [14]:
filtered_known_gene[filtered_known_gene.iloc[:, 18] == "A0A2U3U0J3"]


,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,chr1,65418,71585,ENST00000641515.2_5,0,+,65564,70008,789624,3,...,OR4F5,A0A2U3U0J3,protein_coding,coding,havana_homo_sapiens,protein_coding,"CCDS,Ensembl_canonical,MANE_Select,RNA_Seq_sup...",2,"canonical,basic,all",0


In [16]:
import pandas as pd

canonical_dataset = pd.read_csv("illumina/canonical_dataset_created.txt", sep="\t", header=None)
filtered_known_gene = pd.read_csv("filtered_knownGene_final.txt", sep="\t", header=None)

filtered_known_gene.columns = [
    "chrom", "chromStart", "chromEnd", "name", "score", "strand",
    "thickStart", "thickEnd", "itemRgb", "blockCount",
    "blockSizes", "blockStarts", "other1", "other2", "other3",
    "other4", "other5", "other6", "geneName", "other9",
    "geneType", "typeCategory", "extraInfo", "category",
    "canonicalStatus", "placeholder", "additional"
]

gene_to_verify = "Q6IEY1"  # Replace with desired gene name
canonical_row = canonical_dataset[canonical_dataset[0] == gene_to_verify]
if canonical_row.empty:
    print(f"Gene {gene_to_verify} not found in canonical_dataset_created.txt")
else:
    print("Canonical Dataset Entry:")
    print(canonical_row)

    # Find the same gene in filtered_knownGene_final.txt
    known_gene_row = filtered_known_gene[filtered_known_gene["geneName"] == gene_to_verify]
    if known_gene_row.empty:
        print(f"Gene {gene_to_verify} not found in filtered_knownGene_final.txt")
    else:
        print("\nFiltered KnownGene Entry:")
        print(known_gene_row)

        #convert coordinates for comparison
        known_gene_row["chromStart_1based"] = known_gene_row["chromStart"] + 1
        print("\nConverted Coordinates (1-based):")
        print(known_gene_row[["chrom", "chromStart_1based", "chromEnd", "strand"]])


Canonical Dataset Entry:
            0     1  2          3          4          5          6
2      Q6IEY1  chr1  +     367639     368634     368634     367639
3      Q6IEY1  chr1  -     621095     622034     622034     621095
15442  Q6IEY1  chr5  +  180794268  180795263  180795263  180794268

Filtered KnownGene Entry:
      chrom  chromStart   chromEnd                 name  score strand  \
2      chr1      367639     368634    ENST00000426406.1      0      +   
3      chr1      621095     622034  ENST00000332831.5_7      0      -   
15442  chr5   180794268  180795263  ENST00000456475.1_3      0      +   

       thickStart   thickEnd  itemRgb  blockCount  ...  other6 geneName  \
2          367658     368597   789624           1  ...  OR4F29   Q6IEY1   
3          621095     622034   789624           1  ...  OR4F16   Q6IEY1   
15442   180794287  180795226   789624           1  ...   OR4F3   Q6IEY1   

               other9 geneType                            typeCategory  \
2      prote

/tmp/ipykernel_1318/125858280.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  known_gene_row["chromStart_1based"] = known_gene_row["chromStart"] + 1


In [2]:
input_file = "illumina/shuffled_sequences_with_flanked_Ns.txt"
output_file = "illumina/shuffled_sequences_with_flanked_Ns_reformatted.txt"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    chrom_info = None
    sequence = []

    for line in infile:
        line = line.strip()
        if line.startswith(">"):  
            if chrom_info and sequence:
                outfile.write(f"{chrom_info}\t{''.join(sequence)}\n")
            
            chrom_info = line[1:]  
            sequence = []  
        else: 
            sequence.append(line.strip())
    
    if chrom_info and sequence:
        outfile.write(f"{chrom_info}\t{''.join(sequence)}\n")

print(f"Reformatted file created at {output_file}.")


Reformatted file created at illumina/shuffled_sequences_with_flanked_Ns_reformatted.txt.
